# Major Project Comp8220 _45464006[Praneet Shetty]

## Introduction

This notebook is for the major project submission for COMP8220, on the [language] dataset. It contains the following sections:
<br>
*1:* Data Loading,Data Cleaninig
<br>
*2:* Data pre-processing
<br>
*3:* Machine learning models
<br>
    *3.1:* Conventional Model
    <br>
    *3.2:* Deep learning Model
<br>
*4:* Discussions


Below given gode predicted an Accuracy of <b>59.26</b> submitted as filename"[Public_test_45464006[RF][TFIDF][Stemming][parameter_tuned][Emoji handled]]" for the public test data set which ranked 14 using <b>Random Forest</b> and then on the private test data set the final model predicted an accuracy of <b>64.223 [random Forest]</b> submitted as "[45464006_conventional[Random_forest]_praneetShetty]" which ranked 19 .
<br>
For Conventional Model 4 Conventional models[Naive bayes,Logistic Regression,XGB classifier,Random Forest,SVM] were implemented of which Random Forest was found to be the best models followed by SVM.

For Deep Learning sequential deep learning and RNN using LSTM model were implemented both achieving an accuracy of 58%~60% on the validation data set and 56% on the public test dataset

<b> Final models : </b>

<b> Random Forest :</b> A Random Forest is an ensemble of Decision Trees which are oftern trained via bagging.It is most widely used and is the best classifier algirthms as it introduces randomness when growing trees.It searches for the best features among a random subset of features when splitting a node. It is consider best model as the model achieves Higher bias but lower variance which is essential for the best models.


<b> RNN using LSTM : </b> Recurrent neural networks, of which LSTMs (“long short-term memory” units) are the most powerful and well known subset, are a type of artificial neural network designed to recognize patterns in sequences of data, such as numerical times series data emanating from sensors, stock markets and government agencies (but also including text, genomes, handwriting and the spoken word). What differentiates RNNs and LSTMs from other neural networks is that they take time and sequence into account, they have a temporal dimension.


<b> Performance : </b> All the model were trained on Train+Validation data.Random Forest predicted an accuracy of 60.9% on the validation data set wheras RNN using LSTM predicted an accuracy of () which was lesser than Random Forest. All the other conventional models achieved an accuracy in the range of [52-55%] with random forest achieveing highest Validation accuracy .For Public data set all the models were tested however the highest accuracy was predicted by the random forest which gave [59.27] and RNN using LSTM gave [56 %].

For Final submission on the private test data set,both the submissions were random forest model with submission 1 [TFIDF model ]were trained on train+Public Test+Validation +Private test which gave prediction accuracy [64.153] and for second submission,it was similar but the TFIDF model was trained on [Train+test+validation] which gave higher accuracy [64.223].I didnt applied Deep learning mdoel as fir public test the best accuracy was achieved using random forest.

#### Importing All the required packages and libraries

In [53]:
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
import re

from collections import defaultdict

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from textblob import TextBlob
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

# RNN using LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SpatialDropout1D, LSTM
from keras.callbacks import EarlyStopping

#CNN
from keras.models import Sequential
from keras import layers
from keras.layers import Activation, Dense , Dropout,Bidirectional, GlobalMaxPool1D , LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer 

#Voting classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#import xgboost

np.random.seed(500)

In [3]:
import nltk
nltk.download('wordnet')
from collections import defaultdict
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to C:\Users\Praneet
[nltk_data]     Shetty\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Praneet Shetty\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Data loading

In [4]:
test=np.load('text_test_public_tweets_rand.npy') #Test data(public)
train_label=np.load('text_train_labels.npy') #Train Label Data
train_data=np.load('text_train_tweets.npy')#Train data
val_label=np.load('text_val_labels.npy') #Validation label data
val_data=np.load('text_val_tweets.npy')# Validation data

with open('text_word_to_idx.pkl', 'rb') as f:  #reading the pickle file and storing it for further analysis.
    data = pickle.load(f)


#### Loading the Private test

In [5]:
private_test=np.load('text_test_private_tweets.npy')


#### Defining Function too be used 


In [6]:
#Function for extracting the data in the text format from number format

def get_data(arr,data):
    dat=data
    d=arr
    #df=pd.DataFrame(columns=('Text','Label'))
    idx_to_word = {i: w for w, i in dat.items()}  # interchanging the (Data) ,making it index:word

    l=[] #List for storing all the tweets stored as index into readable text format

    for i in range(len(d)):
        s=''
        for j in range(52): #Range as 52 as all the tweets have 52 indexes.
            a=idx_to_word[d[i][j]] #Mapping the number to respective number
            s= s+' '+a
        l.append(s)
    
    return list(l)


In [7]:
# Converting emojis to words
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
    return text

# Converting emoticons [( :),:( ) ] to words  
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)

    return text


In [8]:
#Function for cleaning the Text data [Removal od special characters,removal of #,]
def clean_data(text):
    processed_features=[]
    t=text
    for i in range(0, len(t)):
    
        # Remove all the special characters
        processed_feature = re.sub(r'[!@$%^&*()_+=#]','', t[i]) #Removes all special character except 
           
        #removing <START>,<USER>,<END>
        processed_feature = re.sub(r'<START>', '', processed_feature)
        processed_feature = re.sub(r'<user>', '', processed_feature)
        processed_feature = re.sub(r'<END>', '', processed_feature)
        processed_feature = re.sub(r'<NULL>', '', processed_feature)
    
        #Removing [<> and other special characters ]
        processed_feature = re.sub(r'[<>?/,.:;]','',processed_feature) #Removes all special character except hashtag
    
    
        #Removing single character
        processed_feature = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature) #Single character
    
        #removing multiple white space with single
        processed_feature = re.sub(r'\s+', ' ', processed_feature)
        #removing white space at the start
        processed_feature = re.sub(r'^\s+', '', processed_feature)
     
        # Converting to Lowercase
        processed_feature = processed_feature.lower()

        processed_features.append(processed_feature)
    return(processed_features)
    

In [9]:
def lemmatization_stemming(df):
    df=df
    
    for index, entry in enumerate(df['Text_token']):
        final_words = []
        word_lemmatized = WordNetLemmatizer()
        porter = PorterStemmer()
        for word, tag in pos_tag(entry):
            if word not in stopwords.words('english'):
                word_final = porter.stem(word_lemmatized.lemmatize(word, tag_map[tag[0]]))
                #print(word_final)
                final_words.append(word_final)
        df.loc[index, 'text_final'] = str(final_words)
        df.loc[index, 'text_final_long'] = str(' '.join(final_words))
    return(df)

### Initialisation
* Data Loading
* Data Cleaning


In [10]:
# Training data
train=pd.DataFrame(columns=('Text','Label'))
lst_train=[]
arr_train=train_data
lst_train=get_data(arr_train,data)


#Validation data
val=pd.DataFrame(columns=('Text','Label'))
lst_val=[]
arr_val=val_data
lst_val=get_data(arr_val,data)

#testing data
lst=[]
arr=test
lst=get_data(arr,data)


#Private test data
lst_pvt=[]
arr_pvt=private_test
lst_pvt=get_data(arr_pvt,data)
    

The code below handles feature extraction by first preprocessing the text, and then extracts feature extraction using TFIDF model

#### Cleaninig of the Train ,Validation  and testing Data

In [33]:
#Training data
processed_features_train = []
processed_features_train= clean_data(lst_train)#Data Cleaning 
train['Text']=processed_features_train
train['Label']=train_label

#Validation
processed_features_val = []
processed_features_val= clean_data(lst_val) #Data Cleaning 
val['Text']=processed_features_val
val['Label']=val_label

#Testing Data
#Testing data [Public testing]
processed_features_test = []
processed_features_test= clean_data(lst) #Data Cleaning 
df_test_clean=pd.DataFrame({'Text':processed_features_test})


#Private test
processed_features_test = []
processed_features_test= clean_data(lst_pvt)
df_Private_test=pd.DataFrame({'Text':processed_features_test})
df_Private_test.head(3)

,Text
0,whatever you decide to do make sure it makes y...
1,accept the challenges so that you can literall...
2,my roommate it ' okay that we can not spell be...


In [34]:
df_test_clean.tail(3)

,Text
16876,the possibilities endless becca when unplug fr...
16877,you have problem yes can you do something abou...
16878,will fight this guy do not insult the lions li...


In [35]:
val.tail(3)

,Text,Label,Text_token,text_final,text_final_long
1457,wont rt things that might offend your am bette...,3,"[wont, rt, things, that, might, offend, your, ...","['wont', 'rt', 'thing', 'might', 'offend', 'go...",wont rt thing might offend good
1458,and after got home in such horrible mood my mo...,3,"[and, after, got, home, in, such, horrible, mo...","['get', 'home', 'horribl', 'mood', 'mom', 'pis...",get home horribl mood mom piss moment step foo...
1459,hit by sudden wave of sadness,3,"[hit, by, sudden, wave, of, sadness]","['hit', 'sudden', 'wave', 'sad']",hit sudden wave sad


In [36]:
train.tail(3)

,Text,Label,Text_token,text_final,text_final_long
7095,vinb am alot more interested in hearing bout d...,3,"[vinb, am, alot, more, interested, in, hearing...","['vinb', 'alot', 'interest', 'hear', 'bout', '...",vinb alot interest hear bout differ parti poli...
7096,overwhelming sadness this too shall pass lost ...,3,"[overwhelming, sadness, this, too, shall, pass...","['overwhelm', 'sad', 'shall', 'pass', 'lose', ...",overwhelm sad shall pass lose lonley startingov
7097,idk why people be glorifying depression would ...,3,"[idk, why, people, be, glorifying, depression,...","['idk', 'peopl', 'glorifi', 'depress', 'would'...",idk peopl glorifi depress would wish real depr...


### Full Data[Train+Validation]

In [12]:
full_data=np.concatenate((train_data,val_data)) #Entire dataset
labe=np.concatenate((train_label,val_label))
labe=labe.tolist()#Combined label
arr_full=full_data
lst_full=get_data(arr_full,data) 

combin_data=np.concatenate((train_data,val_data,test,private_test)) 
arr=combin_data
lst=get_data(arr,data) 

#Full Data
processed_features_full = []
processed_features_full= clean_data(lst_full)#Data Cleaning 
full_data=pd.DataFrame(columns=('Text','Label'))#Storing the dta into dataframe
full_data['Text']=processed_features_full
full_data['Label']=labe

#Combined data[Train+Test_public+Test_private+Validation]
processed_features_comb = []
processed_features_comb= clean_data(lst)#Data Cleaning 
combined_data=pd.DataFrame({'Text':processed_features_comb})#Storing the dta into dataframe


In [16]:
#combined_data.tail()
full_data.tail()


,Text,Label
8553,who can talk to about being with no answer and...,3
8554,brown can induce panic,3
8555,wont rt things that might offend your am bette...,3
8556,and after got home in such horrible mood my mo...,3
8557,hit by sudden wave of sadness,3


### Pre-processing
Test Data ,Validation data and training data are cleaned and ready to be pre-processed,
<br>
For pre-processing following NLP methods were implemented
* Emojis Handling
* Tokenization
* Lemmatisation
* stemming
* Feature Extraction

#### Handling Emojis


In [17]:
# Passing both functions to 'text'
full_data['Text'] = full_data['Text'].apply(convert_emoticons)
full_data['Text'] = full_data['Text'].apply(convert_emojis)

# Passing both functions to 'text'
val['Text'] = val['Text'].apply(convert_emoticons)
val['Text'] = val['Text'].apply(convert_emojis)

# Passing both functions to 'text'
df_test_clean['Text'] = df_test_clean['Text'].apply(convert_emoticons)
df_test_clean['Text'] = df_test_clean['Text'].apply(convert_emojis)

# Passing both functions to 'text'
train['Text'] = train['Text'].apply(convert_emoticons)
train['Text'] = train['Text'].apply(convert_emojis)

In [18]:
df_Private_test['Text'] = df_Private_test['Text'].apply(convert_emoticons)
df_Private_test['Text'] = df_Private_test['Text'].apply(convert_emojis)


In [19]:
for i in range(0, len(full_data)):
    # Remove all the special characters
    full_data['Text'][i] = re.sub(r'[!@$%^&*()_+=#]','', full_data['Text'][i]) #Removes all special character except
        
        
for i in range(0, len(val)):
    # Remove all the special characters
    val['Text'][i] = re.sub(r'[!@$%^&*()_+=#]','', val['Text'][i]) #Removes all special character except 
    
    
for i in range(0, len(train)):
    # Remove all the special characters
    train['Text'][i] = re.sub(r'[!@$%^&*()_+=#]','', train['Text'][i]) #Removes all special character except 
    
for i in range(0, len(df_test_clean)):
    # Remove all the special characters
    df_test_clean['Text'][i] = re.sub(r'[!@$%^&*()_+=#]','', df_test_clean['Text'][i]) #Removes all special character except 

C:\Users\Praneet Shetty\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Praneet Shetty\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Praneet Shetty\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [20]:
for i in range(0, len(df_Private_test)):
    # Remove all the special characters
    df_Private_test['Text'][i] = re.sub(r'[!@$%^''&*()_+=#]','', df_Private_test['Text'][i]) #Removes all special character except
        

### Tokenization

In [21]:
# Tokenise the text
full_data['Text_token']= [word_tokenize(entry) for entry in full_data['Text']]
val['Text_token']= [word_tokenize(entry) for entry in val['Text']]
df_test_clean['Text_token']= [word_tokenize(entry) for entry in df_test_clean['Text']]
train['Text_token']= [word_tokenize(entry) for entry in train['Text']]



In [22]:
df_Private_test['Text_token']= [word_tokenize(entry) for entry in df_Private_test['Text']]


### Lemmatization+stemming

In [24]:
lemmatization_stemming(train)
lemmatization_stemming(val)
lemmatization_stemming(df_test_clean)
lemmatization_stemming(full_data)

,Text,Label,Text_token,text_final,text_final_long
0,it makes me so fucking irate jesus nobody is c...,0,"[it, makes, me, so, fucking, irate, jesus, nob...","['make', 'fuck', 'irat', 'jesu', 'nobodi', 'ca...",make fuck irat jesu nobodi call ppl like hajim...
1,lol adam the bull with his fake outrage,0,"[lol, adam, the, bull, with, his, fake, outrage]","['lol', 'adam', 'bull', 'fake', 'outrag']",lol adam bull fake outrag
2,passed away early this morning in fast and fur...,0,"[passed, away, early, this, morning, in, fast,...","['pass', 'away', 'earli', 'morn', 'fast', 'fur...",pass away earli morn fast furiou style car cra...
3,lol wow was gonna say really haha have you see...,0,"[lol, wow, was, gon, na, say, really, haha, ha...","['lol', 'wow', 'gon', 'na', 'say', 'realli', '...",lol wow gon na say realli haha see chri nah do...
4,need bentobox sushi date riceball spaghetti an...,0,"[need, bentobox, sushi, date, riceball, spaghe...","['need', 'bentobox', 'sushi', 'date', 'ricebal...",need bentobox sushi date ricebal spaghetti oli...
...,...,...,...,...,...
8553,who can talk to about being with no answer and...,3,"[who, can, talk, to, about, being, with, no, a...","['talk', 'answer', 'pay', 'hour', 'work', 'ans...",talk answer pay hour work answer
8554,brown can induce panic,3,"[brown, can, induce, panic]","['brown', 'induc', 'panic']",brown induc panic
8555,wont rt things that might offend your am bette...,3,"[wont, rt, things, that, might, offend, your, ...","['wont', 'rt', 'thing', 'might', 'offend', 'go...",wont rt thing might offend good
8556,and after got home in such horrible mood my mo...,3,"[and, after, got, home, in, such, horrible, mo...","['get', 'home', 'horribl', 'mood', 'mom', 'pis...",get home horribl mood mom piss moment step foo...


In [25]:
lemmatization_stemming(df_Private_test)

,Text,Text_token,text_final,text_final_long
0,whatever you decide to do make sure it makes y...,"[whatever, you, decide, to, do, make, sure, it...","['whatev', 'decid', 'make', 'sure', 'make', 'h...",whatev decid make sure make happi
1,accept the challenges so that you can literall...,"[accept, the, challenges, so, that, you, can, ...","['accept', 'challeng', 'liter', 'even', 'feel'...",accept challeng liter even feel exhilar victor...
2,my roommate it ' okay that we can not spell be...,"[my, roommate, it, ', okay, that, we, can, not...","['roommat', ""'"", 'okay', 'spell', 'autocorrect...",roommat ' okay spell autocorrect terribl first...
3,no but that ' so cute atsu was probably shy ab...,"[no, but, that, ', so, cute, atsu, was, probab...","[""'"", 'cute', 'atsu', 'probabl', 'shi', 'photo...",' cute atsu probabl shi photo cherri help uwu
4,rooneys fucking untouchable is not he been fuc...,"[rooneys, fucking, untouchable, is, not, he, b...","['rooney', 'fuck', 'untouch', 'fuck', 'dread',...",rooney fuck untouch fuck dread depay look dece...
...,...,...,...,...
4252,watch this amazing lively broadcast by lively ...,"[watch, this, amazing, lively, broadcast, by, ...","['watch', 'amaz', 'live', 'broadcast', 'live',...",watch amaz live broadcast live music
4253,the are in contention and hosting nation and c...,"[the, are, in, contention, and, hosting, natio...","['content', 'host', 'nation', 'camden', 'empti...",content host nation camden empti sad
4254,the possibilities endless becca when unplug fr...,"[the, possibilities, endless, becca, when, unp...","['possibl', 'endless', 'becca', 'unplug', ""'"",...",possibl endless becca unplug ' matrix ' stay s...
4255,you have problem yes can you do something abou...,"[you, have, problem, yes, can, you, do, someth...","['problem', 'ye', 'someth']",problem ye someth


In [26]:
lemmatization_stemming(combined_data)
#combined_data

,Text,Text_token,text_final,text_final_long
0,it makes me so fucking irate jesus nobody is c...,"[it, makes, me, so, fucking, irate, jesus, nob...","['make', 'fuck', 'irat', 'jesu', 'nobodi', 'ca...",make fuck irat jesu nobodi call ppl like hajim...
1,lol adam the bull with his fake outrage,"[lol, adam, the, bull, with, his, fake, outrage]","['lol', 'adam', 'bull', 'fake', 'outrag']",lol adam bull fake outrag
2,passed away early this morning in fast and fur...,"[passed, away, early, this, morning, in, fast,...","['pass', 'away', 'earli', 'morn', 'fast', 'fur...",pass away earli morn fast furiou style car cra...
3,lol wow was gonna say really haha have you see...,"[lol, wow, was, gon, na, say, really, haha, ha...","['lol', 'wow', 'gon', 'na', 'say', 'realli', '...",lol wow gon na say realli haha see chri nah do...
4,need 🍱 sushi date 🍙 🍝 an olive guarded date 🧀 ...,"[need, 🍱, sushi, date, 🍙, 🍝, an, olive, guarde...","['need', '🍱', 'sushi', 'date', '🍙', '🍝', 'oliv...",need 🍱 sushi date 🍙 🍝 oliv guard date 🧀 👊 🏼 ro...
...,...,...,...,...
16874,watch this amazing lively broadcast by lively ...,"[watch, this, amazing, lively, broadcast, by, ...","['watch', 'amaz', 'live', 'broadcast', 'live',...",watch amaz live broadcast live music
16875,the are in contention and hosting nation and c...,"[the, are, in, contention, and, hosting, natio...","['content', 'host', 'nation', 'camden', 'empti...",content host nation camden empti sad
16876,the possibilities endless becca when unplug fr...,"[the, possibilities, endless, becca, when, unp...","['possibl', 'endless', 'becca', 'unplug', ""'"",...",possibl endless becca unplug ' matrix ' stay s...
16877,you have problem yes can you do something abou...,"[you, have, problem, yes, can, you, do, someth...","['problem', 'ye', 'someth']",problem ye someth


The Above given code Loads the data and transforms the data into text format 
Following steps were incorporated :
Steps:
* Transformation of data into text format
* Storing the text data into DataFrame[Train,Test,Validation,Full_data,Combined_data]
* Cleaning the text data [Removal of Punctuations,Special characters,Lower case,numbers,Line space]
* Applying pre-processing techniques using NLP tools
    * <b>Tokenization</b>
    * <b>Lemmatization+Stemming </b>
    * <b>Emoji handling[Converts the emoji into word]</b>


### Feature Extraction
For the extracting the feature TFIDF model was implemented.
The TFIDF model was trained on Full data[Train+Public_test+Validation]

#### TFIDF Model.

### TFIDF Model is trained on entire dataset [Train+test+validation]

In [27]:
entire_data=[]
entire_data=list(full_data['text_final_long'])+list(df_test_clean['text_final_long'])

#Training tfidf model
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(max_features=100000,min_df=2, max_df=0.8, stop_words=stopwords.words('english'),ngram_range=(1,3))
tfidf_vect.fit(entire_data)

print(tfidf_vect.vocabulary_)


{'make': 17244, 'fuck': 10048, 'irat': 14234, 'jesu': 14421, 'nobodi': 20003, 'call': 3642, 'ppl': 22938, 'like': 15970, 'abus': 90, 'stop': 28199, 'lmao': 16485, 'make fuck': 17300, 'ppl like': 22943, 'lol': 16523, 'adam': 255, 'bull': 3456, 'fake': 8614, 'outrag': 21378, 'fake outrag': 8616, 'pass': 21670, 'away': 1726, 'earli': 7258, 'morn': 18656, 'fast': 8751, 'furiou': 10282, 'style': 28426, 'car': 3828, 'crash': 5329, 'leav': 15578, 'club': 4563, 'rough': 24951, 'stuff': 28389, 'pass away': 21671, 'away earli': 1735, 'earli morn': 7272, 'fast furiou': 8755, 'car crash': 3829, 'wow': 33443, 'gon': 11524, 'na': 19032, 'say': 25340, 'realli': 23941, 'haha': 12113, 'see': 25697, 'chri': 4341, 'nah': 19095, 'dont': 6866, 'even': 7858, 'snap': 27095, 'anymor': 1178, 'dude': 7183, 'gon na': 11529, 'na say': 19079, 'say realli': 25424, 'dont even': 6869, 'gon na say': 11540, 'need': 19267, 'bentobox': 2494, 'sushi': 28690, 'date': 5785, 'ricebal': 24667, 'spaghetti': 27605, 'oliv': 2099

In [28]:
### Transforming the Dataset as per the TFIDF model
train_tfidf = tfidf_vect.transform(train['text_final_long'])
val_tfidf = tfidf_vect.transform(val['text_final_long'])
test_tfidf = tfidf_vect.transform(df_test_clean['text_final_long'])
full_data_tfidf = tfidf_vect.transform(full_data['text_final_long'])

private_test_tfidf= tfidf_vect.transform(df_Private_test['text_final_long'])
#private_test_tfidf.shape

full_data_tfidf.shape


(8558, 33873)

In [29]:
private_test_tfidf

<4257x33873 sparse matrix of type '<class 'numpy.float64'>'
	with 57596 stored elements in Compressed Sparse Row format>

Thus on training the TFIDF on the entire available data [Train+test+valudation] using Ngram range(1,3) the TFIDF model extracts feature  with 33873 words

<b> TF-IDF :</b> TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.
-monkey learn

# ML Models

### Conventional Model

For Final Submission ,the model used was Random Forest ,the model was trained on TFIDF Transformed data for the entire data [Train+Validation]
<br>
After cross validation following parameters wer considered to be optimal [n_estimators=500,Class_weight = "balanced"]

In [32]:
#Model Difinintion [Random Forest]
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42,class_weight ="balanced")
rnd_clf.fit(full_data_tfidf, labe)

y_pred = rnd_clf.predict(val_tfidf)
print(accuracy_score(val_label, y_pred))

0.6095890410958904


In [ ]:
predictions_RF = rnd_clf.predict(private_test_tfidf)
df=pd.DataFrame(list(predictions_RF))
#df
df.to_csv('45464006_conventional[Random_forest]_praneetShetty.csv')



### Notes on Conventional model

For the Final model,Hyper-parameters were chosen by using cross validation method,
<br>
In addition to the final model I also tried Logistic regression,SVM,Naive Bayes,Decission tree and xgb boost classifier.
<br> All this models accuracy on validation data was achieved to be in the range of 52-55% with Random forest accuracy being consistent in the range of 60.8 %

<br>
On testing this models on the public test data the best model was found to be random forest model with accuracy as 59.2% as compared to others being in the range of 55%.

The Reason being random forest are good in handling Unbalanced data and is great in dealing with high dimensioal data.
With TFIDF other feature extraction methods like [Count Vectoriser,Word2vec] were also implemented however the highest accuracy was achieved in tfidf as it removes the frequent features and least used features .Word2vec feature extraction works best when there is large ammount of data available.

From this conventional model I can conclude that for higher Accuracy data cleaning needs to be done perfectly as on applying [Stemmin,lemmatization and emoji handling ] there was an increase in the accuracy by 8 %.

### Training other models

In [39]:

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
svm_clf = SVC(C=3.0, kernel='poly', degree=3, gamma='scale',decision_function_shape = 'ovo')
nb_clf=MultinomialNB()
xgb_clf = XGBClassifier( n_estimators=1000, nthread= 3,eta= 0.1,min_child_weight= 6,colsample=0.9,colsample_bytree=0.5,subsample=0.9).fit(full_data_tfidf, labe)


from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf,nb_clf,xgb_clf):
    clf.fit(full_data_tfidf, labe)
    y_pred = clf.predict(val_tfidf)
    print(clf.__class__.__name__, accuracy_score(val_label, y_pred))

C:\Users\Praneet Shetty\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression 0.5486301369863014
RandomForestClassifier 0.6095890410958904
SVC 0.6095890410958904
MultinomialNB 0.541095890410959
XGBClassifier 0.5150684931506849


### Cross Validation

In [ ]:
#sigma = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
C = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30]
kernel=['linear', 'poly', 'rbf', 'sigmoid']

errors = list()
sigma_c = list()

for each_c in C:
    for text in kernel:
        clf = svm.SVC(kernel=text, gamma = 1/(2*(each**2)), C=each_c, decision_function_shape='ovr')
        clf.fit(full_data_tfidf, labe)
        errors.append(clf.score(val_tfidf,val_label))
        sigma_c.append((text, each_c))

In [ ]:
index = np.argmax(errors)

sigma_max, c_max = sigma_c[index]

print('The optimal value of sigma is: {}'.format(sigma_max))
print('The optimal value of C is: {}'.format(c_max))

### Deep Learning 

The final model that produced the best performing model was through RNN using LSTM which achieved an acuracy of 56% in the public test submissions.
The input was the raw full data[Train+Validation+Test] pre-proceed using tokenizer  with max words= 7000.
The labels were accordingky formatted using Padding .
The Final model was a sequential model with loss=Categorical cross entropy and optimiser =Adam.the model was trained with epochs as 10 and batch size =20

In [40]:
# RNN using LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SpatialDropout1D, LSTM
from keras.callbacks import EarlyStopping


### Pre-processing +Feature Extraction

In [41]:

tokenizer = Tokenizer(num_words=7000, lower=True)
tokenizer.fit_on_texts(full_data['text_final_long'].values)
word_index = tokenizer.word_index

#print(len(word_index))
X_train = tokenizer.texts_to_sequences(full_data['text_final_long'].values)
X_train = pad_sequences(X_train)
X_train.shape

label1 = pd.get_dummies(labe).values
label1.shape

(8558, 4)

### Model Defining 

In [65]:
#model defining
nn_model = Sequential()
nn_model.add(Embedding(5000, 64, input_length=X_train.shape[1]))
nn_model.add(SpatialDropout1D(0.2))
nn_model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
nn_model.add(LSTM(100, dropout=0.25, recurrent_dropout=0.25))
nn_model.add(Dense(4, activation='softmax'))
nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 20



In [ ]:
#summary
nn_model.summary()


#Fitting the model 
history1 = nn_model.fit(X_train, label1, epochs=epochs, batch_size=batch_size,validation_split=0.25)#callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 12689, 64)         320000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 12689, 64)         0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 12689, 64)         33024     
_________________________________________________________________
lstm_10 (LSTM)               (None, 100)               66000     
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 404       
Total params: 419,428
Trainable params: 419,428
Non-trainable params: 0
_________________________________________________________________


C:\Users\Praneet Shetty\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6418 samples, validate on 2140 samples
Epoch 1/10


##### Validation Accuracy 

In [ ]:
X_vali = tokenizer.texts_to_sequences(val['text_final_long'].values)
X_vali = pad_sequences(X_vali,X_train.shape[1])

label_val = pd.get_dummies(val_label).values
label_val.shape

accr1 = nn_model.evaluate(X_val,label_val)
print(accr1[1])

### Sequential model

In [77]:
vectorizer = CountVectorizer()
combine_data=full_data['Text'].values #gets the data in array format

train_data =train['Text'].values
train_lab=train['Label'].values

val_data =val['Text'].values
val_lab=val['Label'].values

test_data =df_test_clean['Text'].values

vectorizer.fit(combine_data)
X_train=vectorizer.transform(combine_data)
X_val=vectorizer.transform(val_data)

#PAddding the labels 
val_lab=pd.get_dummies(val['Label']).values
train_lab=pd.get_dummies(train['Label']).values
combine_label=pd.get_dummies(full_data['Label']).values
comb_data=vectorizer.transform(combine_data)

In [78]:
test =df_test_clean['Text'].values
val_lab=val['Label'].values
X_test=vectorizer.transform(test)

For this model,
* Sequential model is used as all the layers are stacked sequentially
* Relu an activation function that provides the -ve value as zero is used
* As the output is categorical Softmax is used aso/p layer
* for minimising the error Adaptive Moment Estimation (Adam) is used 


In [69]:
comb_data.shape

(8558, 12689)

In [70]:

input_ =comb_data.shape[1]
model = Sequential()
model.add(layers.Dense(10, input_dim=input_, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', 
             optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 10)                126900    
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 44        
Total params: 126,944
Trainable params: 126,944
Non-trainable params: 0
_________________________________________________________________


In [1]:
import numpy as np
from tensorflow import keras
import tensorflow as tf




Keeping Epochs as 30and batch size as 128 ,Greater thebatch size shorter the execution time but it takes large amount of memory

In [71]:
 history=model.fit(comb_data, combine_label,
                    epochs=30,
                   verbose=True,
                 validation_data=(X_val, val_lab),
                 batch_size=128)

Train on 8558 samples, validate on 1460 samples
Epoch 1/30
8558/8558 [==============================] - 5s 592us/step - loss: 1.3484 - accuracy: 0.3839 - val_loss: 1.3353 - val_accuracy: 0.3877
Epoch 2/30
8558/8558 [==============================] - 2s 248us/step - loss: 1.1714 - accuracy: 0.6407 - val_loss: 1.2533 - val_accuracy: 0.4603
Epoch 3/30
8558/8558 [==============================] - 2s 238us/step - loss: 0.9499 - accuracy: 0.8045 - val_loss: 1.1795 - val_accuracy: 0.4938
Epoch 4/30
8558/8558 [==============================] - 2s 232us/step - loss: 0.7516 - accuracy: 0.8569 - val_loss: 1.1233 - val_accuracy: 0.5144
Epoch 5/30
8558/8558 [==============================] - 2s 230us/step - loss: 0.6040 - accuracy: 0.8715 - val_loss: 1.0788 - val_accuracy: 0.5322
Epoch 6/30
8558/8558 [==============================] - 2s 243us/step - loss: 0.5030 - accuracy: 0.8775 - val_loss: 1.0397 - val_accuracy: 0.5384
Epoch 7/30
8558/8558 [==============================] - 2s 244us/step - loss

### Validation Accuracy

In [72]:

accr = model.evaluate(X_val,val_lab)
print(accr[1])

1460/1460 [==============================] - 0s 341us/step
0.6006849408149719


# Discussion of Model Performance and Implementation
* From the conventional model Random Forest gave the best accuracy and from Deep Learning Model sequential Deep learning model gave the best accuracy for the public test data set . In order to predict labels for public and private dataset training and validation dataset was combined.
* For Feature extraction TFIDF model was trained on entire data [Train+Test+Validation] and then the conventional models were trained on entire data[Train +1validation data]
* Random Forest gave the best accuracy for validation dataset[60.9%] out of all conventional followed by SVM [60.9] and deep learning models [58%]
* For Public dataset the best accuracy obtained was 59.2 which was ranked 14 out of all submission and for Private test dataset the best accuracy obtained was 64.223. Best accuracy obtained for both dataset was done by Random Forest [N_estimaors=500]
* Sequential modelt gave an accuracy of 60.06 for validation dataset.
* For Random Forest accuracy obtained on validation dataset[60.9%]  was less by 3.5% as compared to the accuracy obtained on public dataset[56.94%]
* Concluding that the best Machine learning Model for this task was Random Forest.
